In [1]:
from pickle import load
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add

Using TensorFlow backend.


In [2]:
embedding_matrix = load(open('embedding_matrix_ara.pkl','rb'))

In [3]:
embedding_matrix.shape

(512, 300)

In [6]:
embedding_layer = Embedding(512,
                            300,
                            weights=[embedding_matrix],
                            input_length=120,
                            trainable=False)

In [17]:
#image features
in1 = Input(shape=(1536,))
X1 = Dropout(0.5)(in1)
X1 = Dense(512, activation='relu')(X1)

#sequence model
in2 = Input(shape=(120,))
X2 = embedding_layer(in2)
X2 = Dropout(0.5)(X2)
X2 = LSTM(512)(X2)

# decoder model
D1 = add([X1, X2])
D1 = Dense(1024, activation='relu')(D1)
outputs = Dense(512, activation='softmax')(D1)

model = Model(inputs=[in1, in2], outputs=outputs)

In [18]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [19]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 120)          0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            (None, 1536)         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 120, 300)     153600      input_8[0][0]                    
__________________________________________________________________________________________________
dropout_6 (Dropout)             (None, 1536)         0           input_7[0][0]                    
__________________________________________________________________________________________________
dropout_7 

In [20]:
def gen():
    i = 0
    while True:
        i = i + 1
        x1,x2,y = load(open(('D:\\trainfiles\\train.'+str(i)+'.pkl'),'rb'))
        yield [x1,x2],y
        if i == 2328:
            i = 0

In [ ]:
model.fit_generator(generator=gen(),steps_per_epoch=2328,epochs=8, verbose=2)

Epoch 1/8
